In [1]:
import numpy as np
import json

In [10]:
with open("order_chunks.json", "r") as f:
    chunks = json.load(f)

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [5]:
chunks[0]["chunk_text"]["text"]

'(\n\nprinted page 43895)\n\nExecutive Order 14348\n\nof September 5, 2025\n\nStrengthening Efforts To Protect U.S. Nationals From Wrongful Detention Abroad\n\nBy the authority vested in me as President by the Constitution and the laws of the United States of America, including the Robert Levinson Hostage Recovery and Hostage-Taking Accountability Act (\n\n22 U.S.C. 1741\n\net seq.\n\n) (Levinson Act), it is hereby ordered:'

In [11]:
for chunk in chunks:
  chunk["embedding"] = model.encode(f"search_document: {chunk["chunk_text"]["text"]}")

In [12]:
embeddings = None
n = 0
for chunk in chunks:
    array = np.array(chunk["embedding"], dtype=np.float32).reshape(1, -1)
    if embeddings is None:
        embeddings = array/np.linalg.norm(array)
    else:
        embeddings = np.vstack([embeddings, array/np.linalg.norm(array)])
    n += 1

In [13]:
embeddings.shape[1]

768

In [14]:
import faiss

d = embeddings.shape[1]
base = faiss.IndexFlatL2(d)
index = faiss.IndexIDMap(base)
original_indices = np.arange(len(embeddings), dtype=np.int64)

In [15]:
index.add_with_ids(embeddings, original_indices)

In [16]:
faiss.write_index(index, "orders.index")

In [ ]:
with open("order_embeddings.json", "w") as f:
  json.dump(chunks, f)